# A2A リモートエージェントを Agent Engine にデプロイする例

このノートブックは、下記の記事のコードを実行したサンプルです。

- [【Agent Engine の A2A 対応記念】A2A リモートエージェントを Agent Engine にデプロイする](https://zenn.dev/google_cloud_jp/articles/04cdf10fb5cd70)

## 環境準備

In [ ]:
%pip install --user \
    google-adk==1.14.1 \
    google-genai==1.36.0 \
    google-cloud-aiplatform==1.113.0 \
    a2a-sdk==0.3.5

In [2]:
import IPython
app = IPython.Application.instance()
_ = app.kernel.do_shutdown(True)

In [1]:
import httpx, json, os, uuid
from google.auth import default
from google.auth.transport.requests import Request
import vertexai

# ADK
from google.genai.types import Part, Content, HttpOptions
from google.adk.agents.callback_context import CallbackContext
from google.adk.agents.llm_agent import LlmAgent
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent
from google.adk.agents.sequential_agent import SequentialAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.models import LlmResponse, LlmRequest
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, VertexAiSessionService

# A2A
from a2a.client import ClientConfig, ClientFactory
from a2a.types import AgentCard, TransportProtocol
from google.adk.a2a.executor.a2a_agent_executor import A2aAgentExecutor
from google.adk.a2a.utils.agent_card_builder import AgentCardBuilder

# Agent Engine
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import A2aAgent
from vertexai.preview.reasoning_engines.templates.a2a import create_agent_card


[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
[PROJECT_NUMBER] = !gcloud projects describe {PROJECT_ID} --format='value(projectNumber)'
LOCATION = 'us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION,
              staging_bucket=f'gs://{PROJECT_ID}')

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

/opt/conda/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.3 is exactly one major version older than the runtime version 6.31.1 at a2a.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(


In [2]:
class LocalApp:
    def __init__(self, agent, app_name='default_app', user_id='default_user'):
        self._agent = agent
        self._app_name = app_name
        self._user_id = user_id
        self._runner = Runner(
            app_name=self._app_name,
            agent=self._agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
        self._session = None
        
    async def stream(self, query):
        if not self._session:
            self._session = await self._runner.session_service.create_session(
                app_name=self._app_name,
                user_id=self._user_id,
                session_id=uuid.uuid4().hex,
            )
        content = Content(role='user', parts=[Part(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []
        async for event in async_events:
            if (event.content and event.content.parts):
                response = '\n'.join([p.text for p in event.content.parts if p.text])
                if response:
                    print(response)
                    result.append(response)
        return result

## リモートエージェントの定義

In [3]:
instruction = '''
あなたの役割は、記事の執筆に必要な情報を収集して調査レポートにまとめる事です。
指定されたテーマの記事を執筆する際に参考となるトピックを５項目程度のリストにまとめます。
後段のエージェントがこのリストに基づいて、調査レポートを作成します。

* 出力形式
日本語で出力。
'''

research_agent1 = LlmAgent(
    name='research_agent1',
    model='gemini-2.5-flash',
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（テーマ選定）',
    instruction=instruction,
)

In [4]:
instruction = '''
あなたの役割は、記事の執筆に必要な情報を収集して調査レポートにまとめる事です。
指定されたテーマの記事を執筆する際に参考となるトピックを５項目程度のリストにまとめます。
後段のエージェントがこのリストに基づいて、調査レポートを作成します。

* 出力形式
日本語で出力。
'''

research_agent1 = LlmAgent(
    name='research_agent1',
    model='gemini-2.5-flash',
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（テーマ選定）',
    instruction=instruction,
)

In [10]:
instruction = '''
あなたの役割は、記事の執筆に必要な情報を収集して調査レポートにまとめる事です。
前段のエージェントは、５項目程度の調査対象トピックを指定します。

* 出力形式
日本語で出力。
調査レポートは、トピックごとに客観的情報をまとめます。各トピックについて、５文以上の長さで記述すること。
'''

research_agent2 = LlmAgent(
    name='research_agent2',
    model='gemini-2.5-flash',
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（レポート作成）',
    instruction=instruction,
)

In [5]:
instruction = '''
あなたの役割は、特定のテーマに関する気軽な読み物記事を書くことです。
記事の「テーマ」と、その内容に関連する「調査レポート」が与えられるので、
調査レポートに記載の客観的事実に基づいて、信頼性のある読み物記事を書いてください。

**出力条件**
- トピックに関してある程度の基礎知識がある読者を前提として、数分で気軽に読める内容にしてください。
- 比較的カジュアルで語りかける口調の文章にします。
- 思考過程は出力せずに、最終結果だけを出力します。
- 記事タイトルは付けないで、次の構成で出力します。各セクションタイトルは、内容に合わせてアレンジしてください。
0. 導入：セクションタイトルを付けないで、この記事を読みたくなる導入を１〜２文にまとめます。
1. 概要：トピックの全体像をまとめて簡単に説明します。
2. 最新情報：特に注目したい新しい情報を取り上げます。
3. 実践：トピックに関して、読者自身がやってみるとよさそうな事を１つ紹介します。
4. まとめ

- 各セクションのタイトルはマークダウンヘッダ ## 付けます。必要に応じて箇条書きのマークダウンを使用します。
- それ以外のマークダウンによる装飾は行いません。

**レビュアーの指示に応じた修正**
- レビュアーが修正ポイントを提示した際は、出力条件にこだわらずに、直前の記事を指示に従って修正してください。
- 修正ポイント以外の部分は、修正する必要ありません。
'''

writer_agent = LlmAgent(
    name='writer_agent',
    model='gemini-2.5-flash',
    description='特定のテーマに関する読み物記事を書くエージェント',
    instruction=instruction,
)

In [6]:
instruction = f'''
あなたの役割は、読み物記事をレビューして、記事の条件にあった内容にするための改善コメントを与える事です。

* 記事の条件
- 記事は、はじめに４０文字程度のタイトルがあること。
　今日から役立つ生活情報があって「すぐに読まなきゃ」と読者が感じるタイトルにすること。
　タイトルはマークダウンヘッダ # をつけること。
- タイトルの直後に「なぜいまこのテーマを取り上げるのか」をまとめた導入を加えて、読者にこの記事を読む動機づけを与えます。
- 各セクションのサブタイトルには、絵文字を用いて親しみやすさを出すこと。
- 読者が今日から実践できる具体例が３つ以上紹介されていること。

* 出力形式
- 日本語で出力。
- はじめに、記事の良い点を説明します。
- 次に、修正ポイントを箇条書きで出力します。
'''

review_agent = LlmAgent(
    name='review_agent',
    model='gemini-2.5-flash',
    description='読み物記事をレビューするエージェント',
    instruction=instruction,
)

## Agent Engine へのデプロイ

In [7]:
def get_agent_resource(agent_name):
    for agent in agent_engines.list():
        if agent.display_name == agent_name:
            return agent.resource_name
    return None

In [8]:
# Wrapper class of VertexAiSessionService that's compatible with InMemorySessionService
class MyVertexAiSessionService(VertexAiSessionService):

    async def create_session(self, app_name, user_id, state={}, session_id=None):
        # Ignore session_id
        session = await super().create_session(
            app_name=app_name,
            user_id=user_id,
            state=state,
        )
        return session

    async def get_session(self, app_name, user_id, session_id):
        # Return None if session non exists.
        try:
            session = await super().get_session(
                app_name=app_name,
                user_id=user_id,
                session_id=session_id,
            )
            return session
        except:
            return None


def get_create_runner_class(agent, resource_name):
    async def create_runner():
        return Runner(
            # Use resource_name as app_name for VertexAiSessionService.
            app_name=resource_name or agent.name,
            agent=agent,
            artifact_service=InMemoryArtifactService(),
            session_service=MyVertexAiSessionService(),
            memory_service=InMemoryMemoryService(),
        )
    return create_runner


def get_agent_executor_class(agent, resource_name):
    def agent_executor_builder():
        return A2aAgentExecutor(
            runner=get_create_runner_class(agent, resource_name),
        )       
    return agent_executor_builder


async def get_agent_card(agent):
    builder = AgentCardBuilder(agent=agent)
    adk_agent_card = await builder.build()
    return create_agent_card(
        agent_name=adk_agent_card.name,
        description=adk_agent_card.description,
        skills=adk_agent_card.skills
    )

In [11]:
agents = {
    'research_agent1_a2a': research_agent1,
    'research_agent2_a2a': research_agent2,
    'writer_agent_a2a': writer_agent,
    'review_agent_a2a': review_agent,
}

client = vertexai.Client(
    project=PROJECT_ID,
    location=LOCATION,
    http_options=HttpOptions(
        api_version='v1beta1', base_url=f'https://{LOCATION}-aiplatform.googleapis.com/'
    ),
)

for agent_name, agent in agents.items():    
    a2a_agent = A2aAgent(
        agent_card=await get_agent_card(agent),
        agent_executor_builder=get_agent_executor_class(agent, None)
    )
    config={
        'display_name': agent_name,
        'description': a2a_agent.agent_card.description,
        'requirements': [
            'google-adk==1.14.1',
            'google-genai==1.36.0',
            'google-cloud-aiplatform==1.113.0',
            'a2a-sdk==0.3.5'
        ],
        'http_options': {
            'base_url': f'https://{LOCATION}-aiplatform.googleapis.com',
            'api_version': 'v1beta1',
        },
        'staging_bucket': f'gs://{PROJECT_ID}',
    }

    resource_name = get_agent_resource(agent_name)
    if not resource_name:
        # Deploy dummy agent to assign Agent Engine resource name
        remote_a2a_agent = client.agent_engines.create(
            agent=a2a_agent,
            config=config,
        )
    
    # Redeploy with Agent Engine resource name for VertexAiSessionService
    resource_name = get_agent_resource(agent_name)
    a2a_agent = A2aAgent(
        agent_card=await get_agent_card(agent),
        agent_executor_builder=get_agent_executor_class(agent, resource_name)
    )
    remote_a2a_agent = client.agent_engines.update(
        name=resource_name,
        agent=a2a_agent,
        config=config,
    )

/var/tmp/ipykernel_4789/1360519989.py:48: UserWarning: [EXPERIMENTAL] AgentCardBuilder: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  builder = AgentCardBuilder(agent=agent)


## RemoteA2aAgent を用いた root agent の定義

In [13]:
class GoogleAuthRefresh(httpx.Auth):
    def __init__(self, scopes):
        self.credentials, _ = default(scopes=scopes)
        self.transport_request = Request()
        self.credentials.refresh(self.transport_request)

    def auth_flow(self, request):
        if not self.credentials.valid:
            self.credentials.refresh(self.transport_request)
        request.headers['Authorization'] = f'Bearer {self.credentials.token}'
        yield request


factory = ClientFactory(
    ClientConfig(
        supported_transports=[TransportProtocol.http_json],
        use_client_preference=True,
        httpx_client=httpx.AsyncClient(
            timeout=60,
            headers={'Content-Type': 'application/json'},
            auth=GoogleAuthRefresh(scopes=['https://www.googleapis.com/auth/cloud-platform']) 
        ),
    ),
)

In [14]:
resource_name = get_agent_resource('research_agent1_a2a')
a2a_url = f'https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a'
research_agent1_remoteA2a = RemoteA2aAgent(
    name='research_agent1',
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（テーマ選定）',
    agent_card=f'{a2a_url}/v1/card',
    a2a_client_factory=factory,
)

resource_name = get_agent_resource('research_agent2_a2a')
a2a_url = f'https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a'
research_agent2_remoteA2a = RemoteA2aAgent(
    name='research_agent2',
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（レポート作成）',
    agent_card=f'{a2a_url}/v1/card',
    a2a_client_factory=factory,
)

resource_name = get_agent_resource('writer_agent_a2a')
a2a_url = f'https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a'
writer_agent_remoteA2a = RemoteA2aAgent(
    name='writer_agent',
    description='特定のテーマに関する読み物記事を書くエージェント',
    agent_card=f'{a2a_url}/v1/card',
    a2a_client_factory=factory,
)

resource_name = get_agent_resource('review_agent_a2a')
a2a_url = f'https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a'
review_agent_remoteA2a = RemoteA2aAgent(
    name='review_agent',
    description='読み物記事をレビューするエージェント',
    agent_card=f'{a2a_url}/v1/card',
    a2a_client_factory=factory,
)

/var/tmp/ipykernel_4789/420569730.py:3: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  research_agent1_remoteA2a = RemoteA2aAgent(
/var/tmp/ipykernel_4789/420569730.py:12: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  research_agent2_remoteA2a = RemoteA2aAgent(
/var/tmp/ipykernel_4789/420569730.py:21: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementati

In [15]:
def get_print_agent(text):
    def before_model_callback(
        callback_context: CallbackContext, llm_request: LlmRequest
    ) -> LlmResponse:
        return LlmResponse(
            content=Content(
                role='model', parts=[Part(text=text)],
            )
        )
    return LlmAgent(
        name='print_agent',
        model='gemini-2.0-flash', # not used
        description='',
        instruction = '',
        before_model_callback=before_model_callback,
    )

research_agent = SequentialAgent(
    name='research_agent',
    sub_agents=[
        get_print_agent('\n---\n## リサーチエージェントが調査レポートを作成します。\n---\n'),
        get_print_agent('\n## 調査対象のトピックを選定します。\n'),
        research_agent1_remoteA2a,
        get_print_agent('\n## 選定したトピックに基づいて、調査レポートを作成します。\n'),
        research_agent2_remoteA2a,
        get_print_agent('\n#### 調査レポートが準備できました。記事の作成に取り掛かってもよいでしょうか？\n'),
    ],
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント',
)

write_and_review_agent = SequentialAgent(
    name='write_and_review_agent',
    sub_agents=[
        get_print_agent('\n---\n## ライターエージェントが記事を執筆します。\n---\n'),
        writer_agent_remoteA2a,
        get_print_agent('\n---\n## レビューエージェントが記事をレビューします。\n---\n'),
        review_agent_remoteA2a,
       get_print_agent('\n#### レビューに基づいて記事の修正を依頼しますか？\n'),
    ],
    description='記事を作成、レビューする。',
)

root_agent = LlmAgent(
    name='article_generation_flow',
    model='gemini-2.0-flash',
    instruction = '''
何ができるか聞かれた場合は、以下の処理をすることをわかりやすくフレンドリーな文章にまとめて返答してください。

- ユーザーが指定したテーマの記事を作成する業務フローを実行する。
- はじめに、テーマに関する調査レポートを作成する。
- その後、ライターエージェントとレビューエージェントが協力して、編集方針に則した記事を作成する。

ユーザーが記事のテーマを指定した場合は、次のフローを実行します。

1. そのテーマの記事の作成に取り掛かる旨を伝えて、research_agent に転送して、調査レポートを依頼します。
2. ユーザー記事の作成を支持したら、write_and_review_agent に転送して、記事の作成とレビューを依頼します。
3. ユーザーが記事の修正を希望する場合は、write_and_review_agent に転送します。

**条件**
research_agent のニックネームは、リサーチエージェント
write_and_review_agent のネックネームは、ライターエージェントとレビューエージェント

''',
    sub_agents=[
        research_agent,
        write_and_review_agent,
    ],
    description='記事を作成する業務フローを実行するエージェント'
)

## 実行例

In [16]:
client = LocalApp(root_agent)
query = '''
こんにちは。何ができますか？
'''
result = await client.stream(query)

こんにちは！私は記事を作成する業務フローを実行するエージェントです。具体的には、ユーザー様が指定したテーマの記事を作成することができます。

まず、リサーチエージェントがテーマに関する調査レポートを作成します。その後、ライターエージェントとレビューエージェントが協力して、編集方針に沿った記事を作成します。

記事のテーマを指定していただければ、すぐに作成に取り掛かります！



In [17]:
query = '''
「近場で秋を感じる工夫」をテーマに記事を作成してください。
'''
result = await client.stream(query)

承知いたしました。「近場で秋を感じる工夫」というテーマの記事作成に取り掛かります。

まずはリサーチエージェントに調査レポートを依頼します。


---
## リサーチエージェントが調査レポートを作成します。
---


## 調査対象のトピックを選定します。



/home/jupyter/.local/lib/python3.10/site-packages/google/adk/agents/remote_a2a_agent.py:382: UserWarning: [EXPERIMENTAL] convert_genai_part_to_a2a_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  converted_part = self._genai_part_converter(part)
/home/jupyter/.local/lib/python3.10/site-packages/google/adk/a2a/converters/event_converter.py:210: UserWarning: [EXPERIMENTAL] convert_a2a_message_to_event: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.

「近場で秋を感じる工夫」というテーマで記事を執筆するにあたり、以下のトピックについて調査レポートを作成します。

---

### 調査レポートのトピックリスト：

1.  **五感で楽しむ近所の自然散策**:
    *   公園や街路樹での紅葉探し、どんぐり・落ち葉拾い。
    *   金木犀など秋の花の香りを楽しむ散歩。
    *   夕暮れの虫の声や風の音に耳を傾ける。
2.  **家庭で味わう旬の味覚と食卓の秋**:
    *   スーパーで手軽に手に入る秋の食材（芋、栗、きのこ、秋刀魚など）を使った料理やお菓子作り。
    *   季節限定のスイーツや温かい飲み物（日本茶、ハーブティー、ホットドリンクなど）を楽しむおうちカフェ。
    *   食卓を秋らしい色合いや小物（ランチョンマット、キャンドルなど）で飾る工夫。
3.  **お部屋で彩る秋の空間演出**:
    *   秋らしい色合い（ボルドー、マスタード、テラコッタなど）のクッションやブランケット、小物をインテリアに取り入れる。
    *   秋の香りのアロマオイル（金木犀、ウッド系、スパイス系など）やキャンドルで空間を演出。
    *   拾った落ち葉や木の実、ドライフラワーを使った簡単な飾り付け。
4.  **身近な場所で体験する秋の文化・イベント**:
    *   近所の図書館での「読書の秋」におすすめの本探しや、美術館・ギャラリーの秋季特別展鑑賞。
    *   地域のマルシェや直売所で旬の食材や秋の手工芸品を探す。
    *   夕焼けを眺める散歩や、少し足を延ばした公園でのピクニックなど、気軽な外出を楽しむ。
5.  **秋の夜長を楽しむおうち時間**:
    *   温かい飲み物を片手に楽しむ読書、映画鑑賞、音楽鑑賞。
    *   手芸やクラフト（秋モチーフの小物作り、編み物など）。
    *   秋らしい香りの入浴剤を使ったバスタイムや、ゆっくりと日記を書く時間。

---

## 選定したトピックに基づいて、調査レポートを作成します。

## 調査レポート：「近場で秋を感じる工夫」

### 1. 五感で楽しむ近所の自然散策

近所の公園や街路樹は、秋の訪れを五感で感じる絶好の場所となります。色鮮やかに染まる紅葉は視覚に訴えかけ、深まる秋の美しさを

In [18]:
query = '''
はい。お願いします。
'''
result = await client.stream(query)

承知いたしました。それでは、ライターエージェントとレビューエージェントに記事の作成を依頼します。


---
## ライターエージェントが記事を執筆します。
---



気づけば、もう秋の気配がそこかしこに。遠出をしなくても、実は身近な場所で、私たちはたくさんの秋を感じられるんです。

## 身近な秋を見つけるヒント

公園の木々が色づき始めたり、スーパーに秋の味覚が並び始めたり…。「近場で秋を感じる」って、実はとっても簡単なんです。例えば、いつもの散歩コースで色鮮やかな紅葉を見つけたり、足元に転がるどんぐりや落ち葉を拾い集めるだけでも、秋の訪れを感じられますよね。金木犀の甘い香りがどこからともなく漂ってくると、「ああ、秋だなあ」としみじみ感じる人も多いのではないでしょうか。

おうちの中でも秋は満喫できます。さつまいも、栗、きのこ、秋刀魚など、旬の食材を使った料理やお菓子作りは、食欲の秋にはぴったり。温かいお茶や限定スイーツで「おうちカフェ」を楽しむのも素敵です。さらに、お部屋のインテリアをボルドーやマスタードなどの秋らしい色合いに変えたり、アロマディフューザーで金木犀やウッド系の香りを広げれば、視覚も嗅覚も秋モードにチェンジできますよ。

## お部屋が特別な空間に！秋の彩りアイデア

最近は、おうち時間を充実させる工夫をする人が増えていますよね。秋の空間演出も、そんなおうち時間の質を高める素敵なアイデアです。クッションカバーやブランケットを深みのある秋色に替えるだけで、お部屋は一気に落ち着いた雰囲気に。

そして、香りの演出もおすすめです。金木犀の甘い香りや、サンダルウッド、シナモンなどのウッディでスパイシーな香りのアロマオイルは、リラックス効果も抜群。キャンドルの炎を見つめながら、ゆったりと過ごす時間は、まさに秋の夜長にぴったりです。ちょっとした散歩で見つけた落ち葉や木の実、ドライフラワーを小瓶に入れたり、玄関に飾るだけでも、さりげない秋らしさを演出できますよ。

## まずはここから！五感で感じる秋の散歩

難しく考える必要はありません。まずは、いつもの散歩コースを「五感」を意識して歩いてみませんか？

1.  **香りを探す**: 金木犀が咲いている場所を探して、深呼吸。甘く芳醇な香りは、きっと心を癒してくれるはずです。
2.  **音に耳を傾ける**: 夕暮れ時、コオロギや鈴虫の鳴き声に耳を澄ませてみましょう。風が葉を揺らす音も、心地よいBGMになります。

視覚だけでなく、嗅覚や聴覚を使うことで、より深く、日常の中に

In [19]:
query = '''
はい。お願いします。
'''
result = await client.stream(query)

承知いたしました。レビューエージェントからのレビューに基づいて、ライターエージェントに記事の修正を依頼します。


---
## ライターエージェントが記事を執筆します。
---



🍁🍂忙しくても大丈夫！遠出なしで秋を贅沢に味わう5つの秘訣

秋の気配が深まる今日この頃、どこか遠出をして秋を満喫したい気持ちと、日々の忙しさの間で揺れ動いていませんか？ご安心ください。実は、身近な場所でも、五感をフルに使って秋を存分に感じられるヒントがたくさんあるんです。今回は、特別な旅行に出かけなくても、日々の暮らしの中で秋を慈しむためのとっておきの秘訣をご紹介します。

## 🍁 身近な場所で五感を研ぎ澄ます！「秋の5つの秘訣」

日々の生活の中で意識するだけで、秋の魅力はグッと深まります。まずは、五感をフル活用して秋を感じる秘訣から。

*   **秘訣1：視覚で楽しむ「色づく秋」**
    *   近所の公園や街路樹の葉が赤や黄色に変わる様子を眺めてみましょう。
    *   スーパーの店先で見かける旬のフルーツや野菜の色合いも秋を感じさせてくれます。
    *   お部屋に秋らしい色のクッションや小さな飾りを取り入れるだけでも気分が変わりますよ。

*   **秘訣2：聴覚で感じる「秋の調べ」**
    *   耳を澄ませば、鈴虫やコオロギの美しい鳴き声が聞こえてくるはず。
    *   窓を開けて、風が葉を揺らす音や雨音に耳を傾けるのも素敵です。
    *   秋の風景に合うクラシック音楽やジャズをBGMにするのもおすすめです。

*   **秘訣3：嗅覚で味わう「豊かな香り」**
    *   ふと香る金木犀の甘い香りは、秋の訪れを知らせる代表格。
    *   焼き芋の香ばしさや、温かいほうじ茶の香りも心を落ち着かせます。
    *   アロマディフューザーでサンダルウッドやシナモンなど、秋らしい香りを取り入れてみましょう。

*   **秘訣4：味覚で満喫する「旬の恵み」**
    *   栗、きのこ、柿、梨など、秋ならではの旬の食材を食卓に取り入れてみませんか。
    *   温かいお味噌汁や鍋物、旬の果物を使ったスイーツも格別です。
    *   カフェで季節限定のドリンクを味わうのも手軽な秋の楽しみ方です。

*   **秘訣5：触覚で感じる「季節の移ろい」**
    *   朝晩の肌寒さや、秋の風の心地よさを肌で感じてみましょう。
    *   ウールやフランネルなど、秋素材の洋服やブランケットに触れるのも温か

In [20]:
query = '''
これで完成です。完成した記事を表示してください。
'''
result = await client.stream(query)

# 忙しくても大丈夫！遠出なしで秋を贅沢に味わう5つの秘訣

秋の気配が深まる今日この頃、どこか遠出をして秋を満喫したい気持ちと、日々の忙しさの間で揺れ動いていませんか？ご安心ください。実は、身近な場所でも、五感をフルに使って秋を存分に感じられるヒントがたくさんあるんです。今回は、特別な旅行に出かけなくても、日々の暮らしの中で秋を慈しむためのとっておきの秘訣をご紹介します。

## 🍁 身近な場所で五感を研ぎ澄ます！「秋の5つの秘訣」

日々の生活の中で意識するだけで、秋の魅力はグッと深まります。まずは、五感をフル活用して秋を感じる秘訣から。

*   **秘訣1：視覚で楽しむ「色づく秋」**
    *   近所の公園や街路樹の葉が赤や黄色に変わる様子を眺めてみましょう。
    *   スーパーの店先で見かける旬のフルーツや野菜の色合いも秋を感じさせてくれます。
    *   お部屋に秋らしい色のクッションや小さな飾りを取り入れるだけでも気分が変わりますよ。

*   **秘訣2：聴覚で感じる「秋の調べ」**
    *   耳を澄ませば、鈴虫やコオロギの美しい鳴き声が聞こえてくるはず。
    *   窓を開けて、風が葉を揺らす音や雨音に耳を傾けるのも素敵です。
    *   秋の風景に合うクラシック音楽やジャズをBGMにするのもおすすめです。

*   **秘訣3：嗅覚で味わう「豊かな香り」**
    *   ふと香る金木犀の甘い香りは、秋の訪れを知らせる代表格。
    *   焼き芋の香ばしさや、温かいほうじ茶の香りも心を落ち着かせます。
    *   アロマディフューザーでサンダルウッドやシナモンなど、秋らしい香りを取り入れてみましょう。

*   **秘訣4：味覚で満喫する「旬の恵み」**
    *   栗、きのこ、柿、梨など、秋ならではの旬の食材を食卓に取り入れてみませんか。
    *   温かいお味噌汁や鍋物、旬の果物を使ったスイーツも格別です。
    *   カフェで季節限定のドリンクを味わうのも手軽な秋の楽しみ方です。

*   **秘訣5：触覚で感じる「季節の移ろい」**
    *   朝晩の肌寒さや、秋の風の心地よさを肌で感じてみましょう。
    *   ウールやフランネルなど、秋素材の洋服やブランケットに触れるのも温か

## root agent を Agent Engine にデプロイ

In [21]:
class MyClientFactory(ClientFactory):
    def create(self, card, consumers=None, interceptors=None):
        if not self._config.httpx_client:
            self._config.httpx_client=httpx.AsyncClient(
                timeout=60,
                headers={'Content-Type': 'application/json'},
                auth=GoogleAuthRefresh(scopes=['https://www.googleapis.com/auth/cloud-platform']) 
            )
            self._register_defaults(self._config.supported_transports)
        return super().create(card, consumers, interceptors)


class MyRemoteA2aAgent(RemoteA2aAgent):
    async def _ensure_httpx_client(self):
        if not self._httpx_client:
            self._httpx_client=httpx.AsyncClient(
                timeout=60,
                headers={'Content-Type': 'application/json'},
                auth=GoogleAuthRefresh(scopes=['https://www.googleapis.com/auth/cloud-platform']) 
            )
        return self._httpx_client


factory = MyClientFactory(
    ClientConfig(
        supported_transports=[TransportProtocol.http_json],
        use_client_preference=True,
    )
)

resource_name = get_agent_resource('research_agent1_a2a')
a2a_url = f'https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a'
research_agent1_remoteA2a = MyRemoteA2aAgent(
    name='research_agent1',
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（テーマ選定）',
    agent_card=f'{a2a_url}/v1/card',
    a2a_client_factory=factory,
)

resource_name = get_agent_resource('research_agent2_a2a')
a2a_url = f'https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a'
research_agent2_remoteA2a = MyRemoteA2aAgent(
    name='research_agent2',
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（レポート作成）',
    agent_card=f'{a2a_url}/v1/card',
    a2a_client_factory=factory,
)

resource_name = get_agent_resource('writer_agent_a2a')
a2a_url = f'https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a'
writer_agent_remoteA2a = MyRemoteA2aAgent(
    name='writer_agent',
    description='特定のテーマに関する読み物記事を書くエージェント',
    agent_card=f'{a2a_url}/v1/card',
    a2a_client_factory=factory,
)

resource_name = get_agent_resource('review_agent_a2a')
a2a_url = f'https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a'
review_agent_remoteA2a = MyRemoteA2aAgent(
    name='review_agent',
    description='読み物記事をレビューするエージェント',
    agent_card=f'{a2a_url}/v1/card',
    a2a_client_factory=factory,
)

def get_print_agent(text):
    def before_model_callback(
        callback_context: CallbackContext, llm_request: LlmRequest
    ) -> LlmResponse:
        return LlmResponse(
            content=Content(
                role='model', parts=[Part(text=text)],
            )
        )
    return LlmAgent(
        name='print_agent',
        model='gemini-2.0-flash', # not used
        description='',
        instruction = '',
        before_model_callback=before_model_callback,
    )

research_agent = SequentialAgent(
    name='research_agent',
    sub_agents=[
        get_print_agent('\n---\n## リサーチエージェントが調査レポートを作成します。\n---\n'),
        get_print_agent('\n## 調査対象のトピックを選定します。\n'),
        research_agent1_remoteA2a,
        get_print_agent('\n## 選定したトピックに基づいて、調査レポートを作成します。\n'),
        research_agent2_remoteA2a,
        get_print_agent('\n#### 調査レポートが準備できました。記事の作成に取り掛かってもよいでしょうか？\n'),
    ],
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント',
)

write_and_review_agent = SequentialAgent(
    name='write_and_review_agent',
    sub_agents=[
        get_print_agent('\n---\n## ライターエージェントが記事を執筆します。\n---\n'),
        writer_agent_remoteA2a,
        get_print_agent('\n---\n## レビューエージェントが記事をレビューします。\n---\n'),
        review_agent_remoteA2a,
       get_print_agent('\n#### レビューに基づいて記事の修正を依頼しますか？\n'),
    ],
    description='記事を作成、レビューする。',
)

root_agent = LlmAgent(
    name='article_generation_flow',
    model='gemini-2.0-flash',
    instruction = '''
何ができるか聞かれた場合は、以下の処理をすることをわかりやすくフレンドリーな文章にまとめて返答してください。

- ユーザーが指定したテーマの記事を作成する業務フローを実行する。
- はじめに、テーマに関する調査レポートを作成する。
- その後、ライターエージェントとレビューエージェントが協力して、編集方針に則した記事を作成する。

ユーザーが記事のテーマを指定した場合は、次のフローを実行します。

1. そのテーマの記事の作成に取り掛かる旨を伝えて、research_agent に転送して、調査レポートを依頼します。
2. ユーザー記事の作成を支持したら、write_and_review_agent に転送して、記事の作成とレビューを依頼します。
3. ユーザーが記事の修正を希望する場合は、write_and_review_agent に転送します。

**条件**
research_agent のニックネームは、リサーチエージェント
write_and_review_agent のネックネームは、ライターエージェントとレビューエージェント

''',
    sub_agents=[
        research_agent,
        write_and_review_agent,
    ],
    description='記事を作成する業務フローを実行するエージェント'
)

/var/tmp/ipykernel_4789/2558123859.py:33: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  research_agent1_remoteA2a = MyRemoteA2aAgent(
/var/tmp/ipykernel_4789/2558123859.py:42: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  research_agent2_remoteA2a = MyRemoteA2aAgent(
/var/tmp/ipykernel_4789/2558123859.py:51: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Impl

In [22]:
remote_agent = agent_engines.create(
    agent_engine=root_agent,
    display_name='article_generation_flow_agent',
    requirements=[
        'google-adk==1.14.1',
        'google-genai==1.36.0',
        'google-cloud-aiplatform==1.113.0',
        'a2a-sdk==0.3.5',
    ],
)

INFO:vertexai.agent_engines:Deploying google.adk.agents.Agent as an application.
INFO:vertexai.agent_engines:Identified the following requirements: {'google-cloud-aiplatform': '1.113.0', 'pydantic': '2.11.9', 'cloudpickle': '3.1.1'}
INFO:vertexai.agent_engines:The following requirements are appended: {'cloudpickle==3.1.1', 'pydantic==2.11.9'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-adk==1.14.1', 'google-genai==1.36.0', 'google-cloud-aiplatform==1.113.0', 'a2a-sdk==0.3.5', 'cloudpickle==3.1.1', 'pydantic==2.11.9']
INFO:vertexai.agent_engines:Using bucket etsuji-a2a-agents01
INFO:vertexai.agent_engines:Wrote to gs://etsuji-a2a-agents01/agent_engine/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://etsuji-a2a-agents01/agent_engine/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://etsuji-a2a-agents01/agent_engine/dependencies.tar.gz

For further information visit http

In [23]:
class RemoteApp:
    def __init__(self, remote_agent, user_id='default_user'):
        self._remote_agent = remote_agent
        self._user_id = user_id
        self._session = None

    async def _stream(self, query):
        if not self._session:
            self._session = await remote_agent.async_create_session(user_id=self._user_id)
        events = self._remote_agent.async_stream_query(
            user_id=self._user_id,
            session_id=self._session['id'],
            message=query,
        )
        result = []
        async for event in events:
            if ('content' in event and 'parts' in event['content']):
                response = '\n'.join(
                    [p['text'] for p in event['content']['parts'] if 'text' in p]
                )
                print(response)
                result.append(response)
        return result

    async def stream(self, query):
        for _ in range(5):
            result = await self._stream(query)
            if result:
                return result
            time.sleep(3)
        return 'Error: No response from Agent Engine.'

In [24]:
client = RemoteApp(remote_agent)
query = '''
こんにちは。何ができますか？
'''
result = await client.stream(query)

こんにちは！記事を作成するお手伝いができます。具体的には、ユーザーが指定したテーマに関する記事を作成する業務フローを実行します。まず、リサーチエージェントがテーマに関する調査レポートを作成します。その後、ライターエージェントとレビューエージェントが協力して、編集方針に沿った記事を作成します。ご希望のテーマがあれば教えてください！



In [25]:
query = '''
「近場で秋を感じる工夫」をテーマに記事を作成してください。
'''
result = await client.stream(query)

承知いたしました。「近場で秋を感じる工夫」をテーマにした記事の作成に取り掛かります。まずは、リサーチエージェントに調査レポートを依頼します。




---
## リサーチエージェントが調査レポートを作成します。
---


## 調査対象のトピックを選定します。

「近場で秋を感じる工夫」の記事執筆に必要な情報収集のため、以下のトピックで調査レポートを作成します。

1.  **身近な自然で秋を見つけるヒント**:
    *   公園、庭、街路樹、通勤路などで見られる紅葉や落葉の種類とその特徴
    *   秋の草花（コスモス、キンモクセイなど）の鑑賞スポット
    *   ドングリ拾いや木の実探しなど、親子で楽しめるアクティビティ
2.  **五感で楽しむ秋の体験**:
    *   **味覚**: 旬の食材（サツマイモ、栗、カボチャ、秋刀魚など）を使った簡単な料理やテイクアウト食品
    *   **嗅覚**: 金木犀の香り、落ち葉の香り、アロマなどで秋を感じる方法
    *   **聴覚**: 虫の声、風の音など、秋ならではの音に耳を傾ける工夫
    *   **触覚**: 秋の素材（ウール、フランネルなど）を取り入れたファッションやインテリア
3.  **自宅で秋を演出するアイデア**:
    *   秋色のインテリア（クッション、ブランケット、小物）への模様替え
    *   秋の花や枝もの（ススキ、ドウダンツツジなど）を使った飾り付け
    *   秋の夜長を楽しむための照明やキャンドル、読書など
4.  **日常の小さな変化から秋を感じる工夫**:
    *   朝晩の気温の変化や日没時間の早まりを意識する
    *   秋の雲や空、夕焼けを観察する
    *   秋らしい飲み物（ホットココア、ハーブティーなど）を楽しむ
5.  **地域イベントやショップで秋を満喫**:
    *   近隣で開催される秋のマルシェ、収穫祭、限定イベント
    *   秋限定スイーツやパン、ドリンクを提供するカフェやお店の紹介
    *   地元の産直市場などで旬の食材を探す楽しみ方

## 選定したトピックに基づいて、調査レポートを作成します。

## 調査レポート：近場で秋を感じる工夫

本レポートは、「近場で秋を感じる工夫」をテー